<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [53]:
path = Path('medical-concept-normalization/data_collection')

In [54]:
# training data set

train_csv = path/'TwADR-L.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [55]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,57,hypertrophic scarring
2,84,paraplegic
3,93,shooting
4,97,screwing with my short term memory


In [56]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-0.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,60,cigarette withdrawal
2,108,cramps
3,109,make me drunker
4,133,nightmares


In [57]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [58]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [59]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [60]:
data_clas.add_test(test,label=0)


In [61]:
data_lm.save()
data_clas.save()

In [62]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'drug',
 'by',
 'on',
 'patients',
 'that',
 'as',
 'is',
 'or',
 'at',
 'mg',
 'increased',
 'from',
 'group',
 'mood',
 'study',
 'this',
 'treatment',
 'after',
 'p',
 'release',
 'an',
 'be',
 'me',
 ',',
 'disorder',
 'effect',
 'i',
 'abnormal',
 'decreased',
 'withdrawal',
 'not',
 'than',
 'weight',
 'compared',
 'anxiety',
 'my',
 'are',
 'groups',
 'effects',
 'using',
 'symptoms',
 'no',
 'two',
 'both',
 'drugs',
 'inflammatory',
 'significantly',
 'acute',
 'significant',
 'potassium',
 'skin',
 'day',
 'between',
 'behavior',
 'anti',
 'dose',
 'disorders',
 'time',
 'all',
 'h',
 '(',
 ')',
 'voltaren',
 'used',
 'blood',
 'syndrome',
 'placebo',
 'these',
 'had',
 '-',
 'which',
 'it',
 'results',
 'concentration',
 'efficacy',
 'sleeplessness',
 'diseases',
 'oral',
 'd

In [63]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [64]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [65]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.838095,4.223448,0.346051,17:18


In [66]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.010875,3.832003,0.381310,17:18


In [67]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.724953,3.786204,0.386212,17:35


In [68]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.682380,3.752984,0.389226,17:23


In [69]:
learn.save_encoder('TwADR_pubmed_fold0_first')

In [70]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.148018,6.365398,0.060870,00:24


In [71]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.194335,5.222024,0.086957,00:27


In [72]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.700042,4.914510,0.130435,01:10


In [73]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.436573,4.557035,0.191304,01:07
1,5.217210,4.436341,0.182609,01:10


In [74]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.938648,4.218411,0.217391,01:09
1,4.744634,4.125392,0.252174,01:09


In [75]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.534290,3.940370,0.252174,01:10
1,4.399907,3.865324,0.243478,01:08


In [76]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.154134,3.806175,0.295652,01:09
1,4.035742,3.764087,0.286957,01:05


In [77]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.886497,3.611849,0.313043,01:07
1,3.699626,3.577962,0.313043,01:09


In [78]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.546338,3.582319,0.295652,01:10
1,3.482697,3.530619,0.313043,01:07
2,3.300238,3.451803,0.313043,01:06
3,3.127738,3.455428,0.304348,01:06


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.940164,3.410591,0.321739,01:08
1,2.896303,3.470905,0.321739,01:11
2,2.704823,3.423006,0.330435,01:09
3,2.548885,3.466066,0.330435,01:10


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.450135,3.361786,0.321739,01:08
1,2.389849,3.519087,0.286957,01:09
2,2.213291,3.413743,0.339130,01:07
3,2.058895,3.431944,0.313043,01:09


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.905102,3.434018,0.347826,01:10
1,1.878758,3.483757,0.347826,01:04
2,1.754876,3.493609,0.313043,01:09
3,1.631024,3.518100,0.313043,01:03


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.485999,3.535270,0.347826,01:06
1,1.480606,3.516449,0.365217,01:08
2,1.390557,3.597142,0.347826,01:07
3,1.285815,3.598694,0.339130,01:08


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.162937,3.543431,0.365217,01:09
1,1.166371,3.786370,0.356522,01:02
2,1.102197,3.824914,0.321739,01:04
3,0.992439,3.657404,0.347826,01:10


In [84]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.880399,3.805186,0.339130,01:05
1,0.893807,3.921103,0.330435,01:11
2,0.850358,3.903486,0.365217,01:09
3,0.784890,3.905437,0.347826,01:09


In [85]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.778127,4.036601,0.339130,01:07
1,0.743246,3.961167,0.365217,01:10


In [86]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.685494,3.990064,0.339130,01:11
1,0.667009,4.023195,0.365217,01:05


In [87]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.611870,4.025961,0.339130,01:09


In [88]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold0')

In [89]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.633194,4.026898,0.356522,01:09


In [90]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.550712,4.162331,0.321739,01:14


In [91]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.559730,4.275264,0.347826,01:09


# Part three: Predict on the test dataset

In [92]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [93]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[13, 1597, 265, 1578, 1184, 2149, 1096, 189, 231, 231, 233, 2147, 2147, 233, 249, 249, 1724, 1787, 1373, 1519, 1529, 2002, 644, 749, 749, 749, 452, 792, 792, 792, 813, 852, 852, 749, 452, 2149, 974, 978, 978, 978, 978, 978, 1295, 1822, 1007, 2149, 1669, 1767, 1089, 1884, 426, 441, 1136, 1136, 908, 1681, 1610, 1340, 1942, 1767, 1265, 1930, 1295, 1073, 518, 1308, 1340, 1265, 1340, 1265, 1340, 1340, 1822, 1484, 1508, 1526, 1531, 908, 1597, 1597, 1597, 1597, 1597, 1599, 1096, 1942, 1610, 1099, 1669, 1767, 1682, 1341, 1711, 1545, 1718, 2149, 1718, 1727, 1073, 2149, 1718, 1815, 2149, 1718, 1787, 1787, 2032, 1787, 1787, 1787, 1099, 1798, 1815, 1815, 1767, 1857, 907, 1884, 825, 1787, 1930, 1767, 1936, 815, 1942, 1942, 2002, 2109, 1915, 2018, 1787, 452, 1340, 1877, 249, 1787, 2032, 2147, 2147, 1341, 1787, 1815, 1815]
[13, 60, 108, 109, 133, 154, 187, 189, 231, 231, 233, 233, 233, 233, 249, 249, 265, 393, 418, 520, 561, 561, 644, 749, 749, 749, 772, 792, 792, 792, 813, 852, 852, 887, 908

In [94]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

70
0.48951048951048953
